<a href="https://colab.research.google.com/github/daichira-gif/daichira/blob/main/hindi_hugging_face_dataset_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face データセット作成チュートリアル
本チュートリアルはGENIAC 松尾・岩澤研究室のプロジェクトにて畠山チームが作成したものです。LLMの学習に用いるデータセットをHugging Face Datasets (🤗 Datasets)に登録するためのチュートリアルです。

---

本チュートリアルを通して以下のことを学べます。
- 🤗 Datasetsにリポジトリを作成する方法
- リポジトリにファイルをアップロードする方法
- 大容量ファイルを扱う際に考慮すべきこと

---

🤗 Datasetsには以下の特徴があります
- 様々なフォーマットをサポート（CSV, TSV, JSON, JSON lines, text, Parquet, Arrow, SQLite, WebDataset）
- 様々な圧縮形式をサポート（GZ, BZ2, LZ4, LZMA or ZSTD）
- 利用者は`from datasets import load_dataset`で簡単にデータセットをロードすることが可能
- 高速な読み込みやストリーミングなどの機能の利用が可能

---

大規模言語モデルの学習データをアップロードする場合、以下の点に注意する必要があります
- ファイルサイズが大きい場合、csvやjsonではなくparquetなどの効率的なデータフォーマットを利用することでファイルサイズを抑えることができます。条件により異なりますが、一例として[こちら](https://huggingface.co/datasets/microsoft/orca-math-word-problems-200k/tree/main/data)のファイルではparquetを採用することでcsvやjsonの37%のファイルサイズに抑えています。
- 1ファイルあたりのファイルサイズを抑えることが推奨されています。他のリポジトリをみる限り、おおよそ500MB以内のものが多いようです。
- 大容量のファイルを扱うリポジトリの中にはデータロード用のスクリプト（loading script）を設置しているものがあります。この場合、ユーザーがload_dataset()を実行したとき、リポジトリ作成者が用意したクラウドストレージやオリジナルのデータの保存場所からファイルを取得します。しかしこの手法は非推奨とされており、次のメジャーアップデート以降デフォルトでは使えなくなります（ユーザーが明示的にtruest_remote_code=Trueと設定しなくてはならない）。loading scriptを利用すると、🤗 Datasets上でViewerなどの一部の機能を利用することができなくなり、ユーザーからはデータセットの概要を知ることができません。以上の理由より、本チュートリアルではloading scriptを使用しません。

---

参考
- [Datasets](https://huggingface.co/docs/datasets/v2.18.0/en/index)


---

本チュートリアルに関するご質問・ご指摘は以下にご連絡ください
- [@roy29fuku](https://twitter.com/roy29fuku)

# 基本編


この章では以下の方法を学びます。
- huggingface-cliを利用してHugging Faceにアクセスする方法
- 🤗 Datasetにリポジトリを作成する方法
- リポジトリにデータセットをアップロードする方法
- データセットの読み込み方法

In [ ]:
# 必要なライブラリをインストール
!pip install -qU "huggingface_hub[cli]" datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00


In [ ]:
# Hugging Faceのアカウント名の設定
user_id = input("https://huggingface.co/JINIAC")

https://huggingface.co/JINIAC


In [ ]:
# Hugging Faceに接続
# こちらのページを https://huggingface.co/docs/hub/en/security-tokens を参考に進めてください
# 1. 案内に従いHugging Faceにログインし、Access Tokenを作成（Nameはなんでも良く、RoleはWriteにする）。Tokenをコピーする。
# 2. Colabに戻り「Enter your token」にtokenを入力（Notebookの出力をクリックすると入力できます）
# 3. 「Add token as git credential?」に「n」を入力
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Google Drive へ接続

In [ ]:
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## jsonlファイルの読み込みとアップロード

In [60]:
from datasets import load_dataset


# Google Drive上のファイルパスを指定します。
file_path = '/content/drive/MyDrive/hindi/hin_Deva/massive.jsonl'

# jsonlファイルを読み込みます。
dataset = load_dataset('json', data_files=file_path)

# Hugging Face Datasetsにアップロードします。
dataset.push_to_hub("Hindi_massive", "daichira")


Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/daichira/Hindi_massive/commit/cd36efd09bc1d48eec1c7d73e393c9d10416a633', commit_message='Upload dataset', commit_description='', oid='cd36efd09bc1d48eec1c7d73e393c9d10416a633', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
## データをGoogle Cloud Storageからダウンロードし展開
#!gsutil cp gs://geniac-datasets-tutorial/sample.tar.gz ./sample.tar.gz
#!tar -xf sample.tar.gz

In [ ]:
## ディレクトリの中にtrain.csvがあることを確認
#!ls -lR sample

In [ ]:
## train.csvの内容を確認
#import pandas as pd

#train_df = pd.read_csv("sample/data/train.csv")
#train_df.head(3)

In [61]:
from datasets import Dataset

# データフレームをHugging Face Datasetsの形式に変換します。
#dataset = Dataset.from_pandas(df)

# Hugging Face Datasetsにアップロードします。
dataset.save_to_disk('Hindi_massive')


Saving the dataset (0/1 shards):   0%|          | 0/15115 [00:00<?, ? examples/s]

In [62]:
# データセットをアップロード（該当するリポジトリがない場合は自動で作成される）
# --privatオプションをつけることでprivateリポジトリとして作成される（つけないとpublic）
# 公開範囲を変更する場合はリポジトリのSettings > Change dataset visibilityから変更可能
#!huggingface-cli upload --repo-type=dataset --private sample ./sample .
!huggingface-cli upload --repo-type=dataset --private Hindi_massive

# Organizationに作成する場合は以下のようにリポジトリを記載
#!huggingface-cli upload --repo-type=dataset --private  JINIAC/CleanedParliamentaryProceedings

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
data-00000-of-00001.arrow: 100% 1.47M/1.47M [00:00<00:00, 4.21MB/s]
https://huggingface.co/datasets/daichira/Hindi_massive/tree/main/.


In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset = load_dataset(f"JINIAC/CleanedParliamentaryProceedings")
dataset['train'][0]

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/34 [00:00<?, ?it/s]

{'発言No.': 1,
 '発言者': '山口俊一',
 '発言内容': 'これより会議を開きます。まず、庶務小委員長から報告のため発言を求められておりますので、これを許します。丹羽秀樹君。',
 '国会会期': '212',
 '会議名': '議院運営委員会',
 '会議号': '第14号',
 '会議日付': datetime.datetime(2024, 1, 24, 0, 0)}

# 応用編
この章では以下の方法を学びます。

- 大容量ファイルをリポジトリにアップロードする方法
- 大容量ファイルをロードする方法

---

以下で扱うsample-large.tar.gzにはPMC Open Access Subsetの一部（約40万件）のXMLファイルからAbstractを抽出し、parquet形式にしたファイルが含まれます。各parquetファイルは約100MBで、約10万件の論文に相当するデータが保存されています。全体のデータ数が多い場合は、このようにファイルを分割することで利用者が少量のデータセットを用いて検証することを可能にします（全データをロードする場合と一部のファイルをロードする場合の切り替えが容易です）。

数百GBに及ぶ大きなデータセットを読み込む場合はIterableDatasetを利用することが推奨されています。

In [ ]:
# サンプルデータ（sample-large.tar.gz）をGoogle Driveからダウンロードし展開
!gsutil cp gs://geniac-datasets-tutorial/sample-large.tar.gz ./sample-large.tar.gz
!tar -xf sample-large.tar.gz

In [ ]:
# ディレクトリの中に複数のparquetファイルがあることを確認
!ls -lR sample-large

In [ ]:
# abstracts-xxx.parquetの内容を確認
# parquetはpandasで読み込むことができる
# 大容量ファイルをpandas.DataFrame型にパースしたらto_parquetで変換できる
import pandas as pd

abs_df = pd.read_parquet("sample-large/data/abstracts-001.parquet")
abs_df.head(3)

In [ ]:
# データセットのアップロード（該当するリポジトリがない場合は自動で作成される）
!huggingface-cli upload --quiet --repo-type=dataset --private sample-large ./sample-large .

In [ ]:
# アップロードしたデータセットをロード
# 特に指定しない場合は全てのデータセットがダウンロードされる
# データセットにはDatasetとIterableDatasetがあり、この場合はDatasetとしてロードされる
# Datasetは、indexを指定してデータセット内の標本にアクセスできるが、ディスクかメモリにファイルを保存しなくてはならない
from datasets import load_dataset

dataset = load_dataset(f"{user_id}/sample-large")
print(f'データ数: {dataset["train"].num_rows}')
print(dataset["train"][0])

In [ ]:
# 1ファイルのみをロードする方法
# リポジトリのルートディレクトリからのファイルパスを指定することが可能
dataset = load_dataset(f"{user_id}/sample-large", data_files="data/abstracts-001.parquet")
print(f'データ数: {dataset["train"].num_rows}')

In [ ]:
# 1件ずつストリーミングでロードする方法
# streaming=TrueにすることでIterableDatasetとしてロードできます
# IterableDatasetは数百GB以上のファイルをロードする場合、遅延読み込みができる点や読み込み速度が早い点でDatasetより優れます
# 小さいファイルでは通常通りDatasetに優位性があります
dataset = load_dataset(f"{user_id}/sample-large", streaming=True)
dataset_iter = iter(dataset["train"])

In [ ]:
# 1件ずつデータを取得
next(dataset_iter)

In [ ]:
# 1件ずつデータを取得
next(dataset_iter)